In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
# FINAL dataset  
df = pd.read_csv("D:/traffic-congestion-predictor/data/processed/xbdata.csv", parse_dates=['pickup_hour_dt'])  

   
#df.describe()
#df.info()

In [ ]:
features = [
    'pickup_hour', 'pickup_day_Sunday', 'pickup_day_Monday','pickup_day_Tuesday','pickup_day_Wednesday','pickup_day_Thursday',
    'pickup_day_Friday','pickup_day_Saturday',
    'is_rush_hour', 'is_midweek',
    'PULocationID', 'DOLocationID',
    'PUZone_freq', 'DOZone_freq',
    'PUBorough_freq', 'DOBorough_freq',
    'temp', 'prcp', 'wspd', 'snowed', 'coco'
]
target = 'avg_speed'

X = df[features]
y = df[target]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# Standardization (optional but helps)
# ----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=8, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

xgb_mse = mean_squared_error(y_test, y_pred_xgb)
xgb_rmse=np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, y_pred_xgb)

print(f"XGBoost RMSE: {xgb_rmse:.3f}")
print(f"XGBoost R²: {xgb_r2:.3f}")

In [ ]:
import matplotlib.pyplot as plt

importances = xgb.feature_importances_
feature_names = X.columns
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title("XGBoost Feature Importances")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.tight_layout()
plt.savefig("D:/traffic-congestion-predictor/outputs/plots/xgb_feature_importance.png")
plt.show()



In [ ]:
!pip install joblib

In [ ]:
import joblib
# Save the trained model
joblib.dump(xgb, 'xgboost_congestion_model')